In [ ]:
def output_file(output_file, matrix):
    with open('./Assignment2_Files/output/'+output_file, 'w') as f:
        writer = csv.writer(f, lineterminator='\n')
        for m in matrix:
            writer.writerow(m)

## tf-idf

In [ ]:
import csv
from collections import defaultdict
import numpy as np

In [ ]:
file_name = 'assg2DocSet.csv'

In [ ]:
def count_freq_tf(words):
    frequency = defaultdict(int)
    for token in words:
        frequency[token] += 1
    return dict(frequency)

def count_freq_idf(words):
    frequency = defaultdict(int)
    word_list = []
    for token in words:
        if token in word_list: continue
        frequency[token] += 1
        word_list.append(token)
    return dict(frequency)

# IDFの計算
def calc_idf(n, word_freq):
    return np.log10(n/word_freq)

tfidf = []
tf = []
count_row = 0
all_doc = []

with open('./Assignment2_Files/'+file_name) as f:
    reader = csv.reader(f)

    # idf
    for row in reader:
        all_doc.extend(row)
        count_row += 1
    idf_freq = count_freq_idf(all_doc)
    idf = {}
    for key, value in idf_freq.items():
        idf[key] = calc_idf(count_row, value)

with open('./Assignment2_Files/'+file_name) as f:
    reader = csv.reader(f)

    # tf & tfidf
    for row in reader:
        tf = {}
        fr = count_freq_tf(row)
        freq_max = max(fr.values())
        for key, freq in fr.items():
            tf[key] = freq / freq_max
        buff = []
        for key in range(1, 11):
            if str(key) not in tf.keys():
                buff.append(0.0)
                continue
            buff.append(tf[str(key)]*idf[str(key)])
        tfidf.append(buff)

tfidf = np.round(tfidf, decimals=4)

In [ ]:
output_file('M17335883.csv', tfidf)

## similarity

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
doc = pd.read_csv('./Assignment2_Files/output/M17335883.csv', header=None)
query = pd.read_csv('./Assignment2_Files/query_tfidf.csv', header=None)

In [ ]:
do = doc.values
similarity = []
for q in query.values:
    buff = []
    similarity.append([np.dot(q, d) / (np.linalg.norm(q) * np.linalg.norm(d)) for d in do])

In [ ]:
output_file('R17335883.csv', similarity)

## transition probability

In [ ]:
import numpy as np

In [ ]:
file_name = 'assg2WebLinks.csv'

In [ ]:
adjacency_matrix = np.zeros((5, 5))

with open('./Assignment2_Files/'+file_name) as f:
    reader = csv.reader(f)
    for i, row in enumerate(reader):
        row = [int(r) for r in row]
        for r in row:
            adjacency_matrix[i, r-1] = 1

In [ ]:
probability_matrix = np.zeros((5, 5))

for c in range(len(adjacency_matrix)):
    probability_matrix[:, c] = adjacency_matrix[:, c] / np.sum(adjacency_matrix[:, c])

In [ ]:
np.savetxt('./Assignment2_Files/T17335883.csv', probability_matrix, fmt='%.4f', delimiter=',')

## pagerank

In [ ]:
import numpy.linalg as LA
import pandas as pd

In [ ]:
file_name = 'T17335883.csv'

In [ ]:
probability_matrix = pd.read_csv('./Assignment2_Files/'+file_name, header=None).values

In [ ]:
eig_val, eig_vec = LA.eig(probability_matrix)

In [ ]:
# normalized
for i in range(len(eig_vec)):
    eig_vec[i] = eig_vec[i] / np.linalg.norm(eig_vec[i])

In [ ]:
pagerank = np.round(np.real(eig_vec[:, eig_val.argmax()]), decimals=4)

In [ ]:
with open('./Assignment2_Files/P17335883.csv', 'w') as f:
        writer = csv.writer(f, lineterminator='\n')
        writer.writerow(pagerank)